## Assignment 1


Tasks:  
1. Try to improve the inversion result by applying a slight smoothing to the gradient. 
2. Apply the inversion using different initial model (i.e. different smoothing) and observe the convergence.   
3. Apply a heavy smoothing to the initial model (maybe double the smoothing in the original notebook), then impliment FWI with and without filtering out frequencies below 7 Hz and observe the difference 

For each of the above tasks, explain your observation and the reason for the convergence behavior.  

You can submit your answers in a normal pdf report or using the jupyter notebook in a pdf format but you have to answer the question using Markdown mode.   


In [ ]:
import time
import torch
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import deepwave
from scipy import signal
%matplotlib inline

## setting the parameters 

In [ ]:
device = torch.device('cuda:0')


# Load the true model for forward modelling 
path = './'
velocity_file= path + 'Marm.bin' # true model 

# Define the model and achuisition parameters
par = {'nx':601,   'dx':0.015, 'ox':0,
       'nz':221,   'dz':0.015, 'oz':0,
       'ns':30,    'ds':0.3,   'osou':0,  'sz':0,
       'nr':300,   'dr':0.03,  'orec':0, 'rz':0,
       'nt':4000,  'dt':0.001,  'ot':0,
       'freq':15,
       'num_batches':5, # increase thus number if you have a CUDA out of memory error 
        'FWI_itr': 100
      }

# Mapping the par dictionary to variables 
for k in par:
    locals()[k] = par[k]
    
fs = 1/dt # sampling frequency


# Don't change the below two lines 
num_sources_per_shot=1
num_dims = 2 



## The acquisition set-up

 Create arrays containing the source and receiver locations
 
    x_s: Source locations [num_shots, num_sources_per_shot, num_dimensions].
    
    x_r: Receiver locations [num_shots, num_receivers_per_shot, num_dimensions]

In [ ]:

x_s = torch.zeros(ns, num_sources_per_shot, num_dims)
x_s[:, 0, 1] = torch.arange(ns).float() * ds  
x_s[:, 0, 0] = sz

x_r = torch.zeros(ns, nr, num_dims)
x_r[0, :, 1] = torch.arange(nr).float() * dr
x_r[:, :, 1] = x_r[0, :, 1].repeat(ns, 1)
x_r[:, :, 0] = rz





## Create source wavelet
    [nt, num_shots, num_sources_per_shot]

I use Deepwave's Ricker wavelet function. The result is a normal Tensor - you can use any function to create the wavelet but it needs to be converted to tensor. 

In [ ]:


source_wavelet = (deepwave.wavelets.ricker(freq, nt, dt, 1/freq)
                          .reshape(-1, 1, 1)
                          .repeat(1, ns, num_sources_per_shot))
print(source_wavelet.shape)


plt.plot(np.arange(0,nt)*dt,source_wavelet[:,0,0])
plt.xlabel('Time (s)')

### Load the velocity model 
Note: We often do not update the water layer, thus we will keep it untouched by masking the gradient in this region. 

In [ ]:
# function to get water layer mask
def mask(m):
    water_velocity= 1.5 # km/s
    msk = np.ones_like(m) 
    for i in range(m.shape[0]):
        for j in range (m.shape[1]): 
            msk[i,j] = (0 if m[i,j] <= water_velocity else 1)    
    return msk 


# Load the true model
model_true = (np.fromfile(velocity_file, np.float32)
              .reshape(nz, nx))

msk = mask(model_true) # get the mask for the water layer 

model_true = torch.Tensor(model_true) 
msk = torch.Tensor(msk)



m_vmin, m_vmax = np.percentile(model_true.numpy(), [2,98]) 
plt.imshow(model_true, vmin=m_vmin, vmax=m_vmax, cmap='jet')
plt.title('True');

## Forward modeling 

In [ ]:
# Create 'true' data
prop = deepwave.scalar.Propagator({'vp': model_true.to(device)}, dx) # create a propegator 

data_true = prop(source_wavelet.to(device),
                                x_s.to(device),
                                x_r.to(device), dt).cpu()

In [ ]:
# Plot one shot gather
d_vmin, d_vmax = np.percentile(data_true[:,0].cpu().numpy(), [2,98])

plt.imshow(data_true[:,0,].cpu().numpy(), aspect='auto',
           vmin=-d_vmax, vmax=d_vmax,cmap='gray')



## Create initial model for FWI 



In [ ]:
# Create initial guess model for inversion by smoothing the true model
model_init = gaussian_filter(model_true.cpu().detach().numpy(),
                                           sigma=[10,15])

model_init = torch.Tensor(model_init) 
model_init = model_init * msk  # to preserve the water layer  
model_init[model_init==0] = 1.5 # km/s


# Make a copy so at the end we can see how far we came from the initial model
model = model_init.clone()
model = model.to(device)
model.requires_grad = True

# plot the initial model 
plt.imshow(model_init, vmin=m_vmin, vmax=m_vmax, cmap='jet')
plt.title('initial');

##  Set the optimizer and the criterion 


In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam([{'params': [model], 'lr': 0.01}])

## Filter the low frequencies from the observed data/wavelet   
####  Filter frequency below 7Hz and see how this affects the convergence. You may want to test this with different initial model like in the previous task 
(hint: you can use the below functions to filter the data or whatever function you want).

In [ ]:

# def highpass_filter(freq, wavelet, dt):
#     """
#     Filter out low frequency

#     Parameters
#     ----------
#     freq : :obj:`int`
#     Cut-off frequency
#     wavelet : :obj:`torch.Tensor`
#     Tensor of wavelet
#     dt : :obj:`float32`
#     Time sampling
#     Returns
#     -------
#     : :obj:`torch.Tensor`
#     Tensor of highpass frequency wavelet
#     """

#     sos = signal.butter(4,  6/fs + freq / (0.5 * (1 / dt)), 'hp', output='sos') # AA: Added 6/fs to prevent frequency leak.
#     return torch.tensor( signal.sosfiltfilt(sos, wavelet,axis=0).copy(),dtype=torch.float32)




In [ ]:
# Plot the spectrum 

##

## Main inversion loop 

In [ ]:
# Iterative inversion loop
num_shots_per_batch = int(ns / num_batches)
msk = msk.to(device)
epoch_loss = []
updates = []
gradients = []



t_start = time.time()
for epoch in range(FWI_itr):
  running_loss=0
  for it in range(num_batches):
    optimizer.zero_grad()
    prop = deepwave.scalar.Propagator({'vp': model}, dx)
    batch_src_wvl = source_wavelet[:,it::num_batches,].to(device)
    batch_data_true = data_true[:,it::num_batches].to(device)
    batch_x_s = x_s[it::num_batches].to(device)
    batch_x_r = x_r[it::num_batches].to(device)
    data_pred = prop(batch_src_wvl, batch_x_s, batch_x_r, dt)
    loss = criterion(data_pred, batch_data_true)
    running_loss += loss.item()
    loss.backward()
  epoch_loss.append(running_loss)     

  # Apply some operations to the gradient
  if epoch==0: gmax = torch.abs(model.grad).max()
  model.grad = model.grad / gmax  * msk  # normalizing by the first gradient and mask the wter layer
    

  # update the model 
  optimizer.step()
  print('Epoch:', epoch, 'Loss: ', running_loss)

  # save the model updates and gradients for each iteration
  updates.append(model.detach().clone().cpu().numpy())
  gradients.append(model.grad.cpu().detach().numpy())  
    
  # plotting every 10 itr   
  if epoch % 10 == 0:
    plt.figure(figsize=(8, 4))
    plt.imshow(model.cpu().detach().numpy(), vmin=m_vmin, vmax=m_vmax,
               cmap='jet')
    plt.colorbar(shrink=0.5)
    plt.show()   
#     plt.figure(figsize=(8, 4))
#     gr = model.grad.cpu().detach().numpy()
#     g_min, g_max = np.percentile(gr, [2,98]) 
#     plt.imshow(gr, cmap='bwr',vmin=-g_max,vmax=g_max)
#     plt.colorbar(shrink=0.5)
#     plt.show()
    
t_end = time.time()
print('Runtime:', (t_end - t_start)/60 ,'minutes')

## Plot the convergence curve and the final inversion results 

In [ ]:
updates = np.array(updates)
gradients = np.array(gradients)
obj = np.array(epoch_loss)


In [ ]:
# Plot initial, inverted, and true models
figsize = (8, 4)
plt.figure(figsize=figsize)
plt.imshow(model_init.numpy(), vmin=m_vmin, vmax=m_vmax, cmap='jet')
plt.title('Initial');
plt.figure(figsize=figsize)
plt.imshow(updates[-1], vmin=m_vmin, vmax=m_vmax, cmap='jet')
plt.title('Inverted');
plt.savefig('Inverted_model')
plt.figure(figsize=figsize)
plt.imshow(model_true.numpy(), vmin=m_vmin, vmax=m_vmax, cmap='jet')
plt.title('True');
plt.savefig('True_model')



plt.figure(figsize=(7,4))
plt.plot(np.arange(FWI_itr),obj)
plt.xlabel('iteration#')